In [2]:
from time import sleep
from pathlib import Path

from keystone import Ks, KS_ARCH_PPC, KS_MODE_PPC64
ks = Ks(KS_ARCH_PPC, KS_MODE_PPC64)

import helper_funcs as HF

import dolphin_memory_engine as DME
DME.hook()
assert DME.is_hooked(), "Failed to hook to dolphin"
assert DME.read_bytes(0x80000000,6) == b'GZLE01', "Wrong iso, expecting GZLE01"

In [3]:
# Useful constants
PAD1_addr = 0x803F0F34  # controller 1 C/LR data address
PAD2_addr = 0x803F0F3C  # controller 2 C/LR data address
PAD3_addr = 0x803F0F44  # controller 3 C/LR data address
PAD4_addr = 0x803F0F4C  # controller 4 C/LR data address

nop = bytes.fromhex('60000000')         # "no operation" instruction
button_nop = bytes.fromhex('10808080')  # pscmpu1 cr1, p0, p16 (basically a nop; only affects CR1 which nothing should read from. controller inputs are Start + neutral gray stick)

r12 = 0x803F0F3C                        # where the game jumps when ACE is initially triggered (normally set with Link's X+3 position data)

In [38]:
# Custom DME write function designed to simulate USB Gecko transfers
def my_DME_write(addr, word_bytes, pause=0.001, Nreps=1):
    for _ in range(Nreps):   # perform write Nreps times (if worried about DME writes sometimes not going through)
        DME.write_bytes(addr, word_bytes)
        sleep(pause)   # use to simulate USB Gecko's transfer rate (~1kHz)

In [39]:
########################################################################
# Create phase 1 binary file from file of (address, instruction) pairs
########################################################################
phase1_AI_file = "phase1_addr_instruc_pairs.txt"
phase1_bin_file = "phase1.bin"

phase1_AI_pairs = HF.get_addr_value_pairs_from_files(phase1_AI_file, input_type='ASM', output_type='ASM', ks=ks)
HF.phase1_create_bin_file(phase1_AI_pairs, phase1_bin_file, r12=r12, ks=ks)

In [ ]:
#########################################################################################
# Create phase 2 binary file (main payload) from file of (address, hex) pairs
#########################################################################################
#phase2_AI_file = "phase2_addr_instruc_pairs.txt"
payload_folder = Path.cwd() / 'payload_mods'
phase2_file_list = [AH_file for AH_file in payload_folder.iterdir()]
    
#phase2_AH_file = "phase2_addr_hex_pairs.txt"
phase2_bin_file = "phase2.bin"

#phase2_AH_pairs = HF.get_addr_value_pairs_from_file(phase2_AI_file, input_type='ASM', output_type='hex', ks=ks)
#phase2_AH_pairs = HF.get_addr_value_pairs_from_files(phase2_AH_file, input_type='hex', output_type='hex', ks=ks)

HF.phase2_create_bin_from_files(phase2_file_list, phase2_bin_file, input_type='hex', ks=ks)

In [ ]:
######################################################################################################
# PHASE -1 (pre-ACE): Set controllers 2-4 at start (optional, can manually set before the run instead)
######################################################################################################
# nop out controller 2-4 button/left stick data (will be different if using unplug strats; need to test)
for n in range(3):
    button_addr = 0x803F0F38 + n*0x08
    my_DME_write(button_addr, button_nop)

icbi_r12 = bytes.fromhex('7C0067AC')    # icbi r0, r12 ; invalidates instruction cache at r12=0x803F0F3C (pad 2 C/LR address)
b_42 = bytes.fromhex('4BFFFFF0')        # branch backwards 0x10 bytes

my_DME_write(PAD2_addr, nop)       # clear pad 2 C/LR data
my_DME_write(PAD3_addr, icbi_r12)  # invalidate instruction cache at r12=0x803F0F3C so that the CPU sees updates to pad 2 C/LR data
my_DME_write(PAD4_addr, b_42)      # branch from pad 4 -> pad 2; main loop for phase 1

In [42]:
#########################################################
# PHASE 0.5: nop out all non-essential controller data
#########################################################
# nop out all 4 controllers' button/left stick data (pads 3-4 need to already be harmless before this, but might as well ensure their exact values)
for n in range(4):
    button_addr = 0x803F0F30 + n*0x08
    my_DME_write(button_addr, button_nop, Nreps=10)

# nop out controller 1 C-stick/trigger data (could wait until after phase 1, but might as well do it now)    
my_DME_write(PAD1_addr, nop, Nreps=10)

################################################################################
# PHASE 1: Set up input detection & cache management for phase 2 (main payload)
################################################################################
with open(phase1_bin_file, "rb") as f:
    phase1_input_bytes = f.read()

phase1_instruc_list = [phase1_input_bytes[i:i+4] for i in range(0, len(phase1_input_bytes), 4)]
for phase1_instruc in phase1_instruc_list:
    #print(phase1_instruc)
    my_DME_write(PAD2_addr, phase1_instruc, pause=0.001, Nreps=10)
    

In [43]:
###########################################################
# PHASE 1.5: Transition to phase 2 (main payload)
###########################################################
b_cache2 = bytes.fromhex('48000014')        # 0x803F0F3C: b -> 0x803F0F50
my_DME_write(PAD2_addr, b_cache2, Nreps=3)  # have pad 2 C/LR data branch to the phase 2 cache management module so that pad 3-4 updates work (in phase 1, only pad 2's instruction cache gets refreshed)

my_DME_write(PAD3_addr, nop, Nreps=3)       # clear pad 3 C/LR data (icbi r0, r12) to remove phase 1 cache management 
my_DME_write(PAD4_addr, nop, Nreps=3)       # clear pad 4 C/LR data (b->0x803F0F3C) to remove branch to pad 2
my_DME_write(PAD2_addr, nop, Nreps=3)       # clear pad 2 C/LR data

In [44]:
#############################################
# PHASE 2: Write main payload using pads 1-4
#############################################
with open(phase2_bin_file, "rb") as f:
    phase2_input_bytes = f.read()

instructions = [phase2_input_bytes[i:i+4] for i in range(0, len(phase2_input_bytes), 4)]
for n, instruction in enumerate(instructions):
    pad_idx = n % 4
    pad_address = PAD1_addr + (pad_idx * 0x08)  # controller C-stick/LR address
    my_DME_write(pad_address, instruction)

In [45]:
#########################################################################
# PHASE 3: Perform any cleanup (if necessary) and resume gameplay
#########################################################################
# cleanup TBD; could zero out all addresses in phase1_AI_file but doesn't seem necessary

# Branch to safety to resume game
safety_branch4 = bytes.fromhex('4BE24718')   # 0x803F0F4C: b -> 0x80215664
DME.write_bytes(PAD4_addr, safety_branch4)